# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [116]:
import numpy as np
import pandas as pd
from scipy.stats import binomtest
from scipy.stats import binom
import scipy.stats as stats
from scipy.stats import ttest_1samp
from scipy.stats import t, sem
from statsmodels.stats.proportion import proportions_ztest

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [104]:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')
salaries

Name                              Job Titles  \
0          AARON,  JEFFERY M                                SERGEANT   
1            AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2        AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3        ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4          ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   
...                      ...                                     ...   
33178  ZYLINSKA,  KATARZYNA                           POLICE OFFICER   
33179     ZYMANTAS,  LAURA C                          POLICE OFFICER   
33180      ZYMANTAS,  MARK E                          POLICE OFFICER   
33181    ZYRKOWSKI,  CARLO E                          POLICE OFFICER   
33182   ZYSKOWSKI,  DARIUSZ                  CHIEF DATA BASE ANALYST   

             Department Full or Part-Time Salary or Hourly  Typical Hours  \
0                POLICE                 F           Salary            NaN   
1                POLICE                 F           Salary            NaN   
2      GENERAL SERVICES                 F           Salary            NaN   
3           WATER MGMNT                 F           Salary            NaN   
4                  OEMC                 P           Hourly           20.0   
...                 ...               ...              ...            ...   
33178            POLICE                 F           Salary            NaN   
33179            POLICE                 F           Salary            NaN   
33180            POLICE                 F           Salary            NaN   
33181            POLICE                 F           Salary            NaN   
33182              DoIT                 F           Salary            NaN   

       Annual Salary  Hourly Rate  
0           101442.0          NaN  
1            94122.0          NaN  
2           101592.0          NaN  
3           110064.0          NaN  
4                NaN        19.86  
...              ...          ...  
33178        72510.0          NaN  
33179        48078.0          NaN  
33180        90024.0          NaN  
33181        93354.0          NaN  
33182       115932.0          NaN  

[33183 rows x 8 columns]

Examine the `salaries` dataset using the `head` function below.

In [105]:
salaries.head()

Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary            NaN   
4              OEMC                 P           Hourly           20.0   

   Annual Salary  Hourly Rate  
0       101442.0          NaN  
1        94122.0          NaN  
2       101592.0          NaN  
3       110064.0          NaN  
4            NaN        19.86

We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [106]:
salaries.isna().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [107]:
hourly_vs_salaried = salaries['Salary or Hourly'].value_counts()
hourly_vs_salaried

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [108]:
department_employee = salaries.groupby('Department')['Name'].count()
department_employee

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [109]:
hourly_wages = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
hourly_wages_filled = hourly_wages.fillna(hourly_wages.mean())

test_result = ttest_1samp(hourly_wages_filled, 30)

print("T-Statistic:", test_result.statistic)
print("P-Value:", test_result.pvalue)

T-Statistic: 20.6198057854942
P-Value: 4.3230240486229894e-92


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [110]:
salaried_police = salaries[(salaries['Department'] == 'Police') & (salaries['Salary or Hourly'] == 'Salary')]

if salaried_police.empty:
    print("No data available for salaried Police employees.")
else:
    salaried_police = salaried_police.dropna(subset=['Annual Salary'])

    if len(salaried_police) == 0:
        print("No valid data points after handling missing values.")
    else:
       
        t_statistic, p_value = ttest_1samp(salaried_police['Annual Salary'], 86000)
        one_tailed_p_value = p_value / 2
        hypothesis_test_result = (one_tailed_p_value < 0.05) and (t_statistic > 0)

        print("T-Statistic:", t_statistic)
        print("One-Tailed P-Value:", one_tailed_p_value)
        print("Reject Null Hypothesis:", hypothesis_test_result)

No data available for salaried Police employees.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [111]:
department_hourly_count = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])['Hourly']

most_hourly_workers_department = department_hourly_count.idxmax()

print("Department with the most hourly workers:", most_hourly_workers_department)
print("Number of hourly workers:", department_hourly_count[most_hourly_workers_department])

Department with the most hourly workers: STREETS & SAN
Number of hourly workers: 1862


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [112]:
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

dept_with_most_hourly = hourly_workers['Department'].value_counts().idxmax()

dept_hourly_workers = hourly_workers[hourly_workers['Department'] == dept_with_most_hourly]

t_statistic, p_value = ttest_1samp(dept_hourly_workers['Hourly Rate'].dropna(), 35, alternative='less')

one_tailed_p_value = p_value / 2

hypothesis_test_result = one_tailed_p_value < 0.05

print("T-Statistic:", t_statistic)
print("One-Tailed P-Value:", one_tailed_p_value)
print("Reject Null Hypothesis:", hypothesis_test_result)

T-Statistic: -9.567447887848152
One-Tailed P-Value: 8.344632641176929e-22
Reject Null Hypothesis: True


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [113]:
hourly_wages = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
hourly_wages_filled = hourly_wages.fillna(hourly_wages.mean())

mean_hourly_wage = hourly_wages_filled.mean()
std_error_hourly_wage = sem(hourly_wages_filled.dropna())

deg_freedom = len(hourly_wages_filled) - 1

confidence_interval = t.interval(0.95, deg_freedom, loc=mean_hourly_wage, scale=std_error_hourly_wage)

print("95% Confidence Interval for Mean Hourly Wage:", confidence_interval)

95% Confidence Interval for Mean Hourly Wage: (32.52345834488425, 33.05365708767623)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [114]:
salaried_police = salaries[(salaries['Department'] == 'Police') & (salaries['Salary or Hourly'] == 'Salary')]

if salaried_police['Annual Salary'].notnull().sum() > 0:
    mean_annual_salary_police = salaried_police['Annual Salary'].mean()
    std_error_annual_salary_police = sem(salaried_police['Annual Salary'].dropna())

    df_police = len(salaried_police['Annual Salary']) - 1

    confidence_interval_salary_police = t.interval(0.95, df_police, loc=mean_annual_salary_police, scale=std_error_annual_salary_police)
    print("95% Confidence Interval:", confidence_interval_salary_police)
else:
    print("Insufficient data to calculate the confidence interval.")

Insufficient data to calculate the confidence interval.


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [115]:
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

total_workers = len(salaries)
hourly_workers_count = len(hourly_workers)

null_hypothesis_proportion = 0.25  

z_stat, p_value = proportions_ztest(hourly_workers_count, total_workers, null_hypothesis_proportion)

alpha = 0.05  

print(f'Z-statistic: {z_stat}\nP-value: {p_value}')
if p_value < alpha:
    print("We reject the null hypothesis. The proportion of hourly workers is significantly different from 25%.")
else:
    print("We fail to reject the null hypothesis. The proportion of hourly workers is not significantly different from 25%.")

Z-statistic: -3.5099964213703005
P-value: 0.0004481127249057967
We reject the null hypothesis. The proportion of hourly workers is significantly different from 25%.
